In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/finalised/csv/train_v2.csv')
X_test = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/finalised/csv/test_v2.csv')

In [ ]:
X_train

,CamId,img1,img2,temp1,temp2,temp3,weather1,weather2,weather3
0,623,20130818_201151.jpg,20130818_211149.jpg,30.0,28.9,30.0,sunny,cloudy,sunny
1,8733,20131020_110640.jpg,20131020_160646.jpg,11.0,16.0,15.0,cloudy,cloudy,rainy
2,75,20130426_191257.jpg,20130426_221256.jpg,21.7,28.9,30.0,sunny,sunny,sunny
3,204,20130823_154951.jpg,20130823_181948.jpg,21.7,21.7,27.2,sunny,sunny,sunny
4,10917,20120121_085406.jpg,20120121_095406.jpg,1.0,2.0,2.0,cloudy,rainy,rainy
...,...,...,...,...,...,...,...,...,...
2203,260,20140803_184456.jpg,20140803_191501.jpg,30.0,30.0,30.0,cloudy,cloudy,sunny
2204,3297,20130107_133114.jpg,20130107_170123.jpg,5.0,5.0,5.0,cloudy,cloudy,rainy
2205,260,20130906_141501.jpg,20130906_201451.jpg,13.0,22.0,23.0,sunny,sunny,sunny
2206,7211,20120828_161005.jpg,20120828_164004.jpg,27.0,27.0,26.0,sunny,sunny,sunny


In [ ]:
X_test

,CamId,img1,img2,temp1,temp2,temp3,weather1,weather2,weather3
0,7211,20120526_064041.jpg,20120526_091002.jpg,16.0,16.0,18.0,sunny,sunny,sunny
1,4801,20110327_084003.jpg,20110327_094004.jpg,7.0,8.0,9.0,cloudy,rainy,rainy
2,4801,20111213_111002.jpg,20111213_104048.jpg,1.0,2.0,2.0,cloudy,rainy,rainy
3,75,20141010_174256.jpg,20141010_191301.jpg,25.0,25.0,32.2,sunny,sunny,sunny
4,684,20121105_174154.jpg,20121105_221148.jpg,10.0,12.8,11.7,sunny,sunny,sunny
...,...,...,...,...,...,...,...,...,...
923,5021,20130818_094627.jpg,20130818_124628.jpg,17.0,21.0,20.0,sunny,sunny,sunny
924,11160,20130822_102323.jpg,20130822_132312.jpg,13.0,20.0,23.0,sunny,sunny,sunny
925,4679,20140831_104013.jpg,20140831_101021.jpg,13.0,13.0,14.0,rainy,rainy,rainy
926,623,20120919_221154.jpg,20120919_021147.jpg,26.7,27.2,27.2,sunny,sunny,sunny


In [ ]:
test_df = pd.DataFrame(columns = ['img', 'next_temp'])
train_df = pd.DataFrame(columns = ['img', 'next_temp'])
path = '/content/drive/MyDrive/fyp/fyp2/image/'

for i in range(len(X_test)):
  test_df.loc[len(test_df)] = [str(path) + str(X_test['img1'].iloc[i]), X_test['temp2'].iloc[i]]
  test_df.loc[len(test_df)] = [str(path) + str(X_test['img2'].iloc[i]), X_test['temp3'].iloc[i]]

for i in range(len(X_train)):
  train_df.loc[len(train_df)] = [str(path) + str(X_train['img1'].iloc[i]), X_train['temp2'].iloc[i]]
  train_df.loc[len(train_df)] = [str(path) + str(X_train['img2'].iloc[i]), X_train['temp3'].iloc[i]]


In [ ]:
train_df

,img,next_temp
0,/content/drive/MyDrive/fyp/fyp2/image/20130818...,28.9
1,/content/drive/MyDrive/fyp/fyp2/image/20130818...,30.0
2,/content/drive/MyDrive/fyp/fyp2/image/20131020...,16.0
3,/content/drive/MyDrive/fyp/fyp2/image/20131020...,15.0
4,/content/drive/MyDrive/fyp/fyp2/image/20130426...,28.9
...,...,...
4411,/content/drive/MyDrive/fyp/fyp2/image/20130906...,23.0
4412,/content/drive/MyDrive/fyp/fyp2/image/20120828...,27.0
4413,/content/drive/MyDrive/fyp/fyp2/image/20120828...,26.0
4414,/content/drive/MyDrive/fyp/fyp2/image/20121029...,4.0


In [ ]:
from collections import deque 
import copy 

def filegenerator(df, temporal_length, temporal_stride):
  labels = list(df.next_temp)
  img_list = list(df.img)
  samples = deque()
  sample_count = 0

  for img in img_list:
    samples.append(img)
    if len(samples)== temporal_length: 
      samples_c = copy.deepcopy(samples)
      sample_count += 1

      temp_arr = [[np.float32(labels[0])], [np.float32(labels[1])]]
      yield samples_c, temp_arr
      labels = labels[2:]
      samples.clear()

In [ ]:
def seq_of_frames(df,length,stride):
  file_gen = filegenerator(df,length,stride)
  iterator = True
  data_list = []
  while iterator:
    try:
      X,y = next(file_gen)
      X = list(X) 
      data_list.append([X,y])
    except Exception as e:
      print("An exception has occured:",e)
      iterator = False
  return data_list

In [ ]:
training_data = seq_of_frames(train_df,2,2)
validation_data = seq_of_frames(test_df,2,2)

An exception has occured: 
An exception has occured: 


In [ ]:
#check y shape

training_data[1]

[['/content/drive/MyDrive/fyp/fyp2/image/20131020_110640.jpg',
  '/content/drive/MyDrive/fyp/fyp2/image/20131020_160646.jpg'],
 [[16.0], [15.0]]]

In [ ]:
from keras.utils import data_utils
import keras
import cv2

class DataGenerator(data_utils.Sequence):
  def __init__(self,data,batch_size,dim,is_autoencoder,shuffle):
    #Initializing the values
    self.dim = dim
    self.data  = data
    self.batch_size = batch_size
    self.list_IDs = np.arange(len(data))
    self.is_autoencoder = is_autoencoder
    self.shuffle = shuffle
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indexes = self.list_IDs
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __len__(self):
    return int(np.floor(len(self.data)/self.batch_size))

  def __getitem__(self, index):
    index = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    list_IDs_temp = [self.list_IDs[k] for k in index]
    X,y = self.__data_generation(list_IDs_temp)
    return X,y

  def __data_generation(self,list_IDs_temp):
    X_data = []
    y_data = []
    for i in list_IDs_temp:
      batch_samples = self.data[i][0]
      y = self.data[i][1]
      temp_data_list = []
      for img in batch_samples:
        try:
          image = cv2.imread(img)
          ext_img = cv2.resize(image,self.dim)
          ext_img = cv2.normalize(ext_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
          temp_data_list.append(ext_img)
        except Exception as e:
          print('image: ', img, 'Value error ',e)
          
      if len(temp_data_list) == 2:
        X_data.append(temp_data_list)
        y_data.append(y)

    X = np.array(X_data)
    y = np.array(y_data)
    if self.is_autoencoder == True:
      return X, X
    else:
      return X, y

In [ ]:
params = {
'batch_size':32,
'dim':(320, 180),
'is_autoencoder':False,
'shuffle':False }
train_gen = DataGenerator(training_data,**params)
valid_gen = DataGenerator(validation_data,**params)

In [ ]:
# np.shape(train_gen[0][1])
train_gen[0][0][0]

array([[[[0.9830508 , 0.7754237 , 0.66525424],
         [0.9830508 , 0.7754237 , 0.66525424],
         [0.9788135 , 0.7711865 , 0.66101694],
         ...,
         [0.6779661 , 0.48728812, 0.37288135],
         [0.69491524, 0.5042373 , 0.3898305 ],
         [0.6779661 , 0.48728812, 0.37288135]],

        [[0.9830508 , 0.7754237 , 0.66525424],
         [0.97033894, 0.7627119 , 0.65254235],
         [0.9576272 , 0.75      , 0.6398305 ],
         ...,
         [0.6864407 , 0.4957627 , 0.38135594],
         [0.690678  , 0.5       , 0.38559324],
         [0.6737288 , 0.48305082, 0.36864406]],

        [[0.9830508 , 0.7754237 , 0.66525424],
         [0.96610165, 0.7584746 , 0.6483051 ],
         [0.97457623, 0.7669492 , 0.65677965],
         ...,
         [0.69491524, 0.5042373 , 0.3898305 ],
         [0.6822034 , 0.4915254 , 0.37711865],
         [0.69491524, 0.5042373 , 0.3898305 ]],

        ...,

        [[0.07203389, 0.06779661, 0.11440677],
         [0.22457626, 0.21186441, 0.23728815]

In [ ]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import TimeDistributed 
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import metrics

In [ ]:
def model():
  num_img = 2
  width = 320
  height = 180

  #cnn
  model = Sequential()
  model.add(TimeDistributed(Conv2D(filters=32, kernel_size=3, activation='relu'),
                            input_shape=(num_img, height, width, 3)))
  model.add(TimeDistributed(Conv2D(filters=32, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(MaxPooling2D(pool_size=2)))
  model.add(TimeDistributed(Dropout(0.5)))

  model.add(TimeDistributed(Conv2D(filters=64, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(Conv2D(filters=64, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(MaxPooling2D(pool_size=2)))
  model.add(TimeDistributed(Dropout(0.5)))

  model.add(TimeDistributed(Flatten()))

  #lstm
  model.add(LSTM(64, return_sequences=True, activation='sigmoid'))

  # dense layer
  model.add(Dense(64, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1))

  opt = Adam(learning_rate=0.0001)
  model.compile(loss = 'mse', optimizer=opt, metrics=[metrics.RootMeanSquaredError()])

  return model

In [ ]:
import keras.backend as K

def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))  

In [ ]:
model = model()
filepath = "/content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/ep_{epoch:02d}-val_root_mean_squared_error_{val_root_mean_squared_error:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# no.of img, size,size, channel
model.build((2,180,320,3))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 2, 178, 318, 32)   896       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 176, 316, 32)   9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 2, 88, 158, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 2, 88, 158, 32)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 2, 86, 156, 64)    18496     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 2, 84, 154, 64)    36928     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 2, 42, 77, 64)     0

In [ ]:
history = model.fit(train_gen,epochs=50,validation_data=valid_gen,use_multiprocessing=True,workers=4, callbacks=callbacks_list)

Epoch 1/50
69/69 [==============================] - 89s 1s/step - loss: 392.7008 - root_mean_squared_error: 19.8147 - val_loss: 386.5643 - val_root_mean_squared_error: 19.6612

Epoch 00001: saving model to /content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/ep_01-val_root_mean_squared_error_19.6612.hdf5
Epoch 2/50
69/69 [==============================] - 86s 1s/step - loss: 361.6017 - root_mean_squared_error: 19.0119 - val_loss: 361.1534 - val_root_mean_squared_error: 19.0040

Epoch 00002: saving model to /content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/ep_02-val_root_mean_squared_error_19.0040.hdf5
Epoch 3/50
69/69 [==============================] - 86s 1s/step - loss: 350.2603 - root_mean_squared_error: 18.7145 - val_loss: 332.9749 - val_root_mean_squared_error: 18.2476

Epoch 00003: saving model to /content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/ep_03-val_root_mean_squared_error_18.2476.hdf5
E

In [ ]:
# save history
import pickle
with open('/content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/HistoryDict', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)


In [ ]:
min(history.history['val_root_mean_squared_error'])

4.950204372406006

In [ ]:
accuracy = pd.DataFrame({'train_loss': history.history['loss'], 
                         'train_rmse': history.history['val_root_mean_squared_error'],
                         'validation_loss': history.history['val_loss'],
                         'validation_rmse': history.history['val_root_mean_squared_error']}, 
                        columns = ['train_loss', 'train_rmse', 'validation_loss', 'validation_rmse'])
accuracy 

,train_loss,train_rmse,validation_loss,validation_rmse
0,396.478851,19.661238,386.564270,19.661238
1,372.265656,19.004036,361.153381,19.004036
2,345.870056,18.247601,332.974915,18.247601
3,317.377258,17.385527,302.256531,17.385527
4,286.478912,16.425879,269.809479,16.425879
5,253.286118,15.300998,234.120544,15.300998
6,220.453049,14.200691,201.659637,14.200691
7,190.923370,13.192253,174.035553,13.192253
8,166.244110,12.298211,151.246002,12.298211
9,148.898010,11.680362,136.430847,11.680362


In [ ]:
accuracy.to_csv('/content/drive/MyDrive/fyp/fyp2/finalised/code/benchmarking checkpoint/version4/accuracy.csv', encoding='utf-8', index=False)